In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
train_data = pd.read_csv('training_set_features.csv')
test_data = pd.read_csv('test_set_features.csv')
train_labels_data = pd.read_csv('training_set_labels.csv')


In [4]:
train_data.set_index('respondent_id', inplace=True)
test_data.set_index('respondent_id', inplace=True)
train_labels_data.set_index('respondent_id', inplace=True)

print(train_data.index.equals(train_labels_data.index))  
print(train_data.shape, test_data.shape, train_labels_data.shape)

True
(26707, 35) (26708, 35) (26707, 2)


In [5]:
categorical_cols = [
    'age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status',
    'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
    'employment_industry', 'employment_occupation'
]

numerical_cols = [
    'xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds', 'behavioral_avoidance','behavioral_face_mask', 
    'behavioral_wash_hands', 'behavioral_large_gatherings','behavioral_outside_home', 'behavioral_touch_face', 
    'doctor_recc_xyz','doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months',
    'health_worker', 'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
    'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk',
    'opinion_seas_sick_from_vacc', 'household_adults', 'household_children'
]



categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent value for categorical features
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encode categorical features
])

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean for numerical features
    ('scaler', StandardScaler())  # Standardize numerical features
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ],
    remainder='passthrough'  # Ensure that no columns are dropped
)

X_train = train_data
y_train = train_labels_data
X_test = test_data

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


In [6]:
print(X_train_preprocessed.shape, X_test_preprocessed.shape)
print(y_train.shape)

(26707, 105) (26708, 105)
(26707, 2)


In [8]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [9]:
print(X_train_preprocessed.shape, y_train.shape)

(26707, 105) (26707, 2)


In [10]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_preprocessed, y_train, test_size=0.2, random_state=42)
print(X_train_split.shape, X_val_split.shape, y_train_split.shape, y_val_split.shape)

(21365, 105) (5342, 105) (21365, 2) (5342, 2)


In [11]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_model = MultiOutputClassifier(rf_model, n_jobs=-1)
multi_target_model.fit(X_train_split, y_train_split)

y_val_pred_proba = multi_target_model.predict_proba(X_val_split)
y_val_pred_proba = np.array(y_val_pred_proba)[:, :, 1].T


In [12]:
roc_auc_xyz = roc_auc_score(y_val_split['xyz_vaccine'], y_val_pred_proba[:, 0])
roc_auc_seasonal = roc_auc_score(y_val_split['seasonal_vaccine'], y_val_pred_proba[:, 1])
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2

print(f'ROC AUC value for XYZ Vaccine: {roc_auc_xyz}')
print(f'ROC AUC  value for Seasonal Vaccine: {roc_auc_seasonal}')
print(f'Mean ROC AUC: {mean_roc_auc}')

ROC AUC value for XYZ Vaccine: 0.8639554916841052
ROC AUC  value for Seasonal Vaccine: 0.8570449153459695
Mean ROC AUC: 0.8605002035150373


In [16]:
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=2)
multi_target_model = MultiOutputClassifier(rf_model, n_jobs=2)

param_grid = {
    'estimator__n_estimators': [50, 100],
    'estimator__max_depth': [None, 10],
    'estimator__min_samples_split': [2, 5],
    'estimator__min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(multi_target_model, param_grid, cv=3, scoring='roc_auc', n_jobs=2)
grid_search.fit(X_train_split, y_train_split)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best ROC AUC score from GridSearchCV: {best_score}')

Best parameters: {'estimator__max_depth': None, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 5, 'estimator__n_estimators': 100}
Best ROC AUC score from GridSearchCV: 0.8556272517047927


In [17]:
best_model = grid_search.best_estimator_
y_test_pred_proba = best_model.predict_proba(X_test_preprocessed)

y_test_pred_proba = np.array(y_test_pred_proba)[:, :, 1].T

submission = pd.DataFrame({
    'respondent_id': test_data.index,
    'xyz_vaccine': y_test_pred_proba[:, 0],
    'seasonal_vaccine': y_test_pred_proba[:, 1]
})

submission.to_csv('submission.csv', index=False)
print(submission.head())

   respondent_id  xyz_vaccine  seasonal_vaccine
0          26707     0.199277          0.394498
1          26708     0.040405          0.053210
2          26709     0.270790          0.707270
3          26710     0.545662          0.893416
4          26711     0.258517          0.505339
